In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


from multiprocessing import Pool,freeze_support
import multiprocessing
import itertools as itrt
import time

In [2]:
multiprocessing.cpu_count()

4

In [2]:
def func(x):
    return x**2

In [4]:

pool = Pool(processes=4)


In [7]:
def f(x):
    time.sleep(1)
    value = x * x
    print('{}s passed...\t{}\t(pid:{})'.format(int(time.time() - start), value, os.getpid()))
    return value

In [ ]:
if __name__ == '__main__':
    result = Pool(4).map(f, [1,2,3])
    print(result)

In [4]:
if __name__ == '__main__':
    freeze_support()
    pool = Pool(processes=4)
    print(pool.map(func, *range(10)))

TypeError: map() takes from 3 to 4 positional arguments but 12 were given

In [ ]:
def tomap(args):
    return getattr(args[0], args[1])(*args[2:])

In [5]:
result = Pool(4).map_async(func, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [8]:
print(result.get(10))

TimeoutError: 

In [1]:
X =[1,2,3,4,5]

In [3]:
X[2]

3

In [ ]:
print(Pool(4).map(func, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))

In [6]:
if __name__ == '__main__':
    with Pool(4) as p:
        print(p.map_async(func, [1, 2, 3]))

In [10]:
apool = pool.apply_async(func, [3])

In [12]:
apool

TypeError: 'ApplyResult' object is not callable

In [2]:
train = pd.read_csv(r'C:\Users\ISHIKAWA\Kaggle\Titanic\train.csv')
test = pd.read_csv(r'C:\Users\ISHIKAWA\Kaggle\Titanic\test.csv')

In [3]:
def names(train, test):
    for i in [train, test]:
        i['Name_Len'] = i['Name'].apply(lambda x: len(x))
        i['Name_Title'] = i['Name'].apply(lambda x: x.split(',')[1]).apply(lambda x: x.split()[0])
        del i['Name']
    return train, test

In [4]:
def age_impute(train, test):
    for i in [train, test]:
        i['Age_Null_Flag'] = i['Age'].apply(lambda x: 1 if pd.isnull(x) else 0)
        data = train.groupby(['Name_Title', 'Pclass'])['Age']
        i['Age'] = data.transform(lambda x: x.fillna(x.mean()))
    return train, test

In [5]:
def fam_size(train, test):
    for i in [train, test]:
        i['Fam_Size'] = np.where((i['SibSp']+i['Parch']) == 0 , 'Solo',
                           np.where((i['SibSp']+i['Parch']) <= 3,'Nuclear', 'Big'))
        del i['SibSp']
        del i['Parch']
    return train, test

In [6]:
def ticket_grouped(train, test):
    for i in [train, test]:
        i['Ticket_Lett'] = i['Ticket'].apply(lambda x: str(x)[0])
        i['Ticket_Lett'] = i['Ticket_Lett'].apply(lambda x: str(x))
        i['Ticket_Lett'] = np.where((i['Ticket_Lett']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), i['Ticket_Lett'],
                                   np.where((i['Ticket_Lett']).isin(['W', '4', '7', '6', 'L', '5', '8']),
                                            'Low_ticket', 'Other_ticket'))
        i['Ticket_Len'] = i['Ticket'].apply(lambda x: len(x))
        del i['Ticket']
    return train, test

In [7]:
def cabin(train, test):
    for i in [train, test]:
        i['Cabin_Letter'] = i['Cabin'].apply(lambda x: str(x)[0])
        del i['Cabin']
    return train, test

In [8]:
def cabin_num(train, test):
    for i in [train, test]:
        i['Cabin_num1'] = i['Cabin'].apply(lambda x: str(x).split(' ')[-1][1:])
        i['Cabin_num1'].replace('an', np.NaN, inplace = True)
        i['Cabin_num1'] = i['Cabin_num1'].apply(lambda x: int(x) if not pd.isnull(x) and x != '' else np.NaN)
        i['Cabin_num'] = pd.qcut(train['Cabin_num1'],3)
    train = pd.concat((train, pd.get_dummies(train['Cabin_num'], prefix = 'Cabin_num')), axis = 1)
    test = pd.concat((test, pd.get_dummies(test['Cabin_num'], prefix = 'Cabin_num')), axis = 1)
    del train['Cabin_num']
    del test['Cabin_num']
    del train['Cabin_num1']
    del test['Cabin_num1']
    return train, test

In [9]:
def embarked_impute(train, test):
    for i in [train, test]:
        i['Embarked'] = i['Embarked'].fillna('S')
    return train, test

In [10]:
test['Fare'].fillna(train['Fare'].mean(), inplace = True)

In [11]:
def dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett', 'Cabin_Letter', 'Name_Title', 'Fam_Size']):
    for column in columns:
        train[column] = train[column].apply(lambda x: str(x))
        test[column] = test[column].apply(lambda x: str(x))
        good_cols = [column+'_'+i for i in train[column].unique() if i in test[column].unique()]
        train = pd.concat((train, pd.get_dummies(train[column], prefix = column)[good_cols]), axis = 1)
        test = pd.concat((test, pd.get_dummies(test[column], prefix = column)[good_cols]), axis = 1)
        del train[column]
        del test[column]
    return train, test

In [12]:
def drop(train, test, bye = ['PassengerId']):
    for i in [train, test]:
        for z in bye:
            del i[z]
    return train, test

In [13]:
def mean_encoding(train, test, cols = ['Pclass', 'Sex', 'Embarked', 'Cabin_Letter', 'Name_Title', 'Fam_Size']):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    skf = skf.split(train, train['Survived'])
    for col in cols:
        train[col+'_mean_encoding'] = 0
        test[col+'_mean_encoding'] = test[col].map(train.groupby(col)['Survived'].mean())
    for tr_idx,val_idx in skf:
        X_tr, X_val = train.loc[tr_idx], train.loc[val_idx]
        for col in cols:
            means = X_val[col].map(X_tr.groupby(col)['Survived'].mean())
            train.loc[val_idx, col+'_mean_encoding'] = means
    prior = train['Survived'].mean()
    train.fillna(prior, inplace=True)
    test.fillna(prior, inplace=True)
    return train,test

In [14]:
train, test = names(train, test)
train, test = age_impute(train, test)
train, test = cabin_num(train, test)
train, test = cabin(train, test)
train, test = embarked_impute(train, test)
train, test = fam_size(train, test)
test['Fare'].fillna(train['Fare'].mean(), inplace = True)
train, test = ticket_grouped(train, test)
train, test = mean_encoding(train, test, cols = ['Pclass', 'Sex', 'Embarked',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = dummies(train, test, columns = ['Pclass', 'Sex', 'Embarked', 'Ticket_Lett',
                                                                     'Cabin_Letter', 'Name_Title', 'Fam_Size'])
train, test = drop(train, test)

In [15]:
pd.set_option('display.max_columns',100)
train.head()

,Survived,Age,Fare,Name_Len,Age_Null_Flag,"Cabin_num_(1.999, 28.667]","Cabin_num_(28.667, 65.667]","Cabin_num_(65.667, 148.0]",Ticket_Len,Pclass_mean_encoding,Sex_mean_encoding,Embarked_mean_encoding,Cabin_Letter_mean_encoding,Name_Title_mean_encoding,Fam_Size_mean_encoding,Pclass_3,Pclass_1,Pclass_2,Sex_male,Sex_female,Embarked_S,Embarked_C,Embarked_Q,Ticket_Lett_A,Ticket_Lett_P,Ticket_Lett_S,Ticket_Lett_1,Ticket_Lett_3,Ticket_Lett_2,Ticket_Lett_C,Ticket_Lett_Low_ticket,Ticket_Lett_Other_ticket,Cabin_Letter_n,Cabin_Letter_C,Cabin_Letter_E,Cabin_Letter_G,Cabin_Letter_D,Cabin_Letter_A,Cabin_Letter_B,Cabin_Letter_F,Name_Title_Mr.,Name_Title_Mrs.,Name_Title_Miss.,Name_Title_Master.,Name_Title_Rev.,Name_Title_Dr.,Name_Title_Ms.,Name_Title_Col.,Fam_Size_Nuclear,Fam_Size_Solo,Fam_Size_Big
0,0,22.0,7.2500,23,0,0,0,0,9,0.255102,0.193548,0.342857,0.314079,0.159036,0.576923,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1,1,38.0,71.2833,51,0,0,0,1,8,0.648352,0.752000,0.547445,0.607843,0.797980,0.587234,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,1,26.0,7.9250,22,0,0,0,0,16,0.236041,0.742063,0.349237,0.295495,0.694444,0.306636,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
3,1,35.0,53.1000,44,0,0,0,1,6,0.639053,0.742063,0.349237,0.590909,0.792079,0.585586,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
4,0,35.0,8.0500,24,0,0,0,0,6,0.236041,0.188312,0.349237,0.295495,0.159036,0.306636,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [16]:
test.describe()

,Age,Fare,Name_Len,Age_Null_Flag,"Cabin_num_(1.999, 28.667]","Cabin_num_(28.667, 65.667]","Cabin_num_(65.667, 148.0]",Ticket_Len,Pclass_mean_encoding,Sex_mean_encoding,Embarked_mean_encoding,Cabin_Letter_mean_encoding,Name_Title_mean_encoding,Fam_Size_mean_encoding,Pclass_3,Pclass_1,Pclass_2,Sex_male,Sex_female,Embarked_S,Embarked_C,Embarked_Q,Ticket_Lett_A,Ticket_Lett_P,Ticket_Lett_S,Ticket_Lett_1,Ticket_Lett_3,Ticket_Lett_2,Ticket_Lett_C,Ticket_Lett_Low_ticket,Ticket_Lett_Other_ticket,Cabin_Letter_n,Cabin_Letter_C,Cabin_Letter_E,Cabin_Letter_G,Cabin_Letter_D,Cabin_Letter_A,Cabin_Letter_B,Cabin_Letter_F,Name_Title_Mr.,Name_Title_Mrs.,Name_Title_Miss.,Name_Title_Master.,Name_Title_Rev.,Name_Title_Dr.,Name_Title_Ms.,Name_Title_Col.,Fam_Size_Nuclear,Fam_Size_Solo,Fam_Size_Big
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,28.435654,35.619000,27.483254,0.205742,0.069378,0.069378,0.074163,6.875598,0.392771,0.390046,0.396935,0.376812,0.392205,0.392206,0.521531,0.255981,0.222488,0.636364,0.363636,0.645933,0.244019,0.110048,0.031100,0.078947,0.078947,0.153110,0.306220,0.227273,0.071770,0.035885,0.016746,0.782297,0.083732,0.021531,0.002392,0.031100,0.016746,0.043062,0.019139,0.574163,0.172249,0.186603,0.050239,0.004785,0.002392,0.002392,0.004785,0.346890,0.605263,0.047847
std,13.333010,55.840751,9.971228,0.404727,0.254400,0.254400,0.262349,2.821437,0.166287,0.266400,0.090442,0.152335,0.283478,0.139391,0.500135,0.436934,0.416416,0.481622,0.481622,0.478803,0.430019,0.313324,0.173797,0.269980,0.269980,0.360525,0.461475,0.419572,0.258417,0.186227,0.128474,0.413179,0.277317,0.145320,0.048912,0.173797,0.128474,0.203240,0.137177,0.495062,0.378049,0.390059,0.218700,0.069088,0.048912,0.048912,0.069088,0.476551,0.489380,0.213698
min,0.830000,0.000000,13.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.242363,0.188908,0.339009,0.299854,0.000000,0.161290,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,7.895800,20.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.242363,0.188908,0.339009,0.299854,0.156673,0.303538,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,28.724891,14.454200,25.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.242363,0.188908,0.339009,0.299854,0.156673,0.303538,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,36.000000,31.500000,30.750000,0.000000,0.000000,0.000000,0.000000,8.000000,0.629630,0.742038,0.389610,0.299854,0.697802,0.578767,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0

In [17]:
train.shape

(891, 51)

In [18]:
cabin_num = {'Cabin_num_(1.999, 28.667]':'Cabin_num1', 'Cabin_num_(28.667, 65.667]':'Cabin_num2',\
             'Cabin_num_(65.667, 148.0]':'Cabin_num3',}

train = train.rename(columns = cabin_num)
test = test.rename(columns = cabin_num)

In [19]:
X_test = test

In [20]:
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]

rf = RandomForestClassifier(criterion='gini', 
                             n_estimators=700,
                             min_samples_split=10,
                             min_samples_leaf=1,
                             max_features='auto',
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1)
rf.fit(X_train, y_train)
print("%.4f" % rf.oob_score_)

0.8384


In [21]:
NN = NearestNeighbors(n_neighbors=5)
NN.fit(X_train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [22]:
neighs = NN.kneighbors(X_train)[1][0]

In [23]:
np.where(y_train[neighs]==y_train[neighs].iloc[0],1,2)

array([1, 1, 1, 1, 1])

In [24]:
np.array(y_train[neighs])[1]

0

In [25]:
np.bincount(y_train[neighs])/5

array([ 1.])

In [27]:
#np.bincount(neighs_y[:k], minlength = self.n_classes_)/k

In [28]:
sum(np.bincount(y_train[neighs])/5).astype('int')

1

In [29]:
type(len(np.bincount(y_train[neighs])/5))

int

In [ ]:
count = 1
for i in range(len(y_train[neighs])):
    if y_train[neighs].iloc[i] == y_train[neighs].iloc[i+1]:
        count +=1
    else:
        break

In [ ]:
neighs = NN.kneighbors(X_train)[1][2]

In [ ]:
np.where(y_train[neighs]==np.array(y_train[neighs])[0])

In [ ]:
neighs = y_train[NN.kneighbors(X_train)[1][6]]

In [ ]:
y_train[neighs]

In [ ]:
NN.kneighbors(X_train)[0][6][0]-NN.kneighbors(X_train)[0][6][-1]

In [ ]:
NN.kneighbors(X_train)[0][6][1]-NN.kneighbors(X_train)[0][6][0]

In [30]:
NN.kneighbors(X_train)[1][2]

array([  2, 216, 884,  45, 611], dtype=int64)

In [32]:
NN.kneighbors(X_train)[0][2]

array([ 0.        ,  1.00037597,  2.71318259,  4.24717487,  4.33432692])

In [31]:
NN.kneighbors(X_train)[0][2][np.where(y_train[NN.kneighbors(X_train)[1][2]]==0)[0][0]]

2.7131825938030256

In [ ]:
np.where(y_train[NN.kneighbors(X_train)[1][0]]==1,y_train[NN.kneighbors(X_train)[1][0]],-1)==-1

In [ ]:
np.where(y_train[NN.kneighbors(X_train)[1][2]]==1)[0][0]

In [ ]:
np.bincount(y_train[NN.kneighbors(X_train)[1][6]][:3], minlength=2)

In [ ]:
NN.kneighbors(X_train)[0][2][np.where(y_train[NN.kneighbors(X_train)[1][2]]==1)[0]].mean()

In [ ]:
len(y_train[NN.kneighbors(X_train)[1][2]][:3]/sum(y_train[NN.kneighbors(X_train)[1][2]][:3]))

In [ ]:
len(np.bincount(y_train[NN.kneighbors(X_train)[1][6]][:5], minlength=2)/sum(np.bincount(y_train[NN.kneighbors(X_train)[1][6]][:5], minlength=2)))

In [44]:
len(np.where(y_train[NN.kneighbors(X_train)[1][0]]==1)[0])

0

In [54]:
NN.kneighbors(X_train)[0][2]

array([ 0.        ,  1.00037597,  2.71318259,  4.24717487,  4.33432692])

In [ ]:
np.where(neighs_y[:k]==c)[0][0]

In [1]:
def func(x):
    return x*x
func(3)

9

In [2]:
from multiprocessing import Pool
pool = Pool(processes=4)


In [ ]:
if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(func, [1, 2, 3]))

In [ ]:
pool.map(func, [3])

In [ ]:

eta = [0.03, 0.1, 0.3, 1,]
subsample = [0.2,0.4,0.6,0.8,1]
colsample_bytree = [0.1,0.15,0.2,0.3,0.5,0.7,0.9]
max_depth = [3,4,5,6,7,8,9,10]
l2_lambda = [0.01,0.03,0.1,0.3,1,3,10]
n_estimators = [50, 100, 400, 700, 1000]

tuned_parameters = {'learning_rate':eta, 'subsample':subsample, 'n_estimators':n_estimators,\
                        'colsample_bytree':colsample_bytree, 'reg_lambda':l2_lambda, 'max_depth':max_depth}

gbm = RandomizedSearchCV(
    xgb.XGBClassifier(), # 識別器
    tuned_parameters, # 最適化したいパラメータセット
    n_iter = 20, 
    n_jobs = 4,
    cv=5, # 交差検定の回数
    scoring='accuracy') # モデルの評価関数の指定


gbm.fit(X_train, y_train)
print(gbm.best_score_)
print(gbm.best_params_)

In [ ]:
model_xgb = xgb.XGBClassifier(n_estimators = 1000, max_depth = 6,\
                       learning_rate = 0.03, subsample = 0.8,\
                       colsample_bytree = 0.15, reg_lambda = 10)

In [ ]:
num_leaves = [10*i for i in range (1,10)]
n_estimators = [50*i for i in range(1,8)]
max_depth = [i for i in range(1,10)]
learning_rate = [0.05, 0.1, 0.2, 0.3]
subsample = [0.6,0.7,0.8,0.9]
subsample_freq = [10,15,20]
colsample_bytree = [0.6,0.7,0.8,0.9]
reg_lambda = [10**i for i in range(3)]


tuned_parameters = dict(num_leaves = num_leaves, n_estimators = n_estimators, max_depth = max_depth,\
                       learning_rate = learning_rate, subsample = subsample, subsample_freq = subsample_freq,\
                       colsample_bytree = colsample_bytree, reg_lambda = reg_lambda)


lgb_clf = RandomizedSearchCV(
        lgb.LGBMClassifier(), # 識別器
        tuned_parameters, # 最適化したいパラメータセット 
        cv=5, # 交差検定の回数
        n_iter = 20,
        scoring ='accuracy') # モデルの評価関数の指定


lgb_clf.fit(X_train, y_train)
print(lgb_clf.best_score_)
print(lgb_clf.best_params_)

In [ ]:
model_lgb = lgb.LGBMClassifier(num_leaves = 80, n_estimators = 150, max_depth = 8,\
                       learning_rate = 0.2, subsample = 0.8, subsample_freq = 10,\
                       colsample_bytree = 0.9, reg_lambda = 10)

In [21]:
from sklearn.preprocessing import RobustScaler, StandardScaler

In [22]:
r_scaler = RobustScaler()
r_scaler.fit(X_train)
X_train_r = r_scaler.transform(X_train)
X_test_r = r_scaler.transform(X_test)

s_scaler = StandardScaler()
s_scaler.fit(X_train_r)
X_train_s = s_scaler.transform(X_train_r)
X_test_s = s_scaler.transform(X_test_r)


In [23]:
from sklearn.model_selection import GridSearchCV

C = [10**i for i in range(4)]

tuned_parameters = dict(C = C)


svc = GridSearchCV(
        SVC(), # 識別器
        tuned_parameters, # 最適化したいパラメータセット 
        cv=5, # 交差検定の回数
        scoring ='accuracy') # モデルの評価関数の指定


svc.fit(X_train_s, y_train)
print(svc.best_score_)
print(svc.best_params_)

0.840628507295
{'C': 1}


In [24]:
model_svc = SVC(C=1)

In [25]:
from sklearn.ensemble import VotingClassifier

vote = VotingClassifier(estimators = [('rf',rf),('svc',model_svc)], voting = 'hard')
vote.fit(X_train_s,y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_w...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None)

In [26]:
preds = vote.predict(X_test_s)
preds

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0,

In [27]:
test_sub = pd.read_csv(r'C:\Users\ISHIKAWA\Kaggle\Titanic\test.csv')
submission = pd.DataFrame({
        "PassengerId": test_sub["PassengerId"],
        "Survived": preds
    })
submission.to_csv(r'C:\Users\ISHIKAWA\Kaggle\Titanic\submission_89.csv', index=False)